<a href="https://colab.research.google.com/github/Musavvir25/First-website/blob/main/Mu11savvir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Load dataset
file_path = '/content/Mobile Price Prediction Datatset.csv'  # Update the correct path
data = pd.read_csv(file_path)

# Fill missing values with column mean
data.fillna(data.mean(numeric_only=True), inplace=True)

# Select numeric features
data = data.select_dtypes(include=['number'])

# Define features (X) and target variable (y)
X = data.drop(columns=['Price'], errors='ignore')  # Ensure 'Price' is the target
y = data['Price'].values.reshape(-1, 1)  # Modify if your target column has a different name

# Normalize features and target using MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)  # Normalized between 0 and 1

# Split dataset into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the Improved Neural Network Model
class MobilePriceANN(nn.Module):
    def __init__(self, input_size):
        super(MobilePriceANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)  # Regression output

        self.leaky_relu = nn.LeakyReLU(0.01)
        self.dropout = nn.Dropout(0.3)  # Regularization

        # Apply Xavier Initialization
        for layer in [self.fc1, self.fc2, self.fc3, self.fc4, self.fc5]:
            nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        x = self.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.fc3(x))
        x = self.leaky_relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Initialize model
input_size = X_train.shape[1]  # Number of features
model = MobilePriceANN(input_size)

# Define loss function (MSE + MAE) and optimizer
criterion_mse = nn.MSELoss()
criterion_mae = nn.L1Loss()  # MAE
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)  # Reduce LR every 20 epochs

# Training loop
epochs = 300  # Increased epochs for better learning
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train_tensor)

    loss_mse = criterion_mse(outputs, y_train_tensor)
    loss_mae = criterion_mae(outputs, y_train_tensor)
    loss = 0.7 * loss_mse + 0.3 * loss_mae  # Weighted combination of MSE and MAE

    loss.backward()
    optimizer.step()
    scheduler.step()  # Adjust learning rate

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)

# Convert predictions back to original scale
y_pred = scaler_y.inverse_transform(predictions.numpy())
y_actual = scaler_y.inverse_transform(y_test_tensor.numpy())

# Final error calculation
final_mse = mean_squared_error(y_actual, y_pred)
print("Final Mean Squared Error:", final_mse)

# Save the trained model
torch.save(model.state_dict(), "mobile_price_prediction_optimized.pth")

# Load & Use the Model for Future Predictions
model.load_state_dict(torch.load("mobile_price_prediction_optimized.pth"))
model.eval()

with torch.no_grad():
    test_predictions = model(X_test_tensor)

# Convert back from MinMaxScaler and show sample predictions
test_predictions = scaler_y.inverse_transform(test_predictions.numpy())
print("Sample Predictions:", test_predictions[:5].flatten().tolist())


Epoch [10/300], Loss: 0.191177
Epoch [20/300], Loss: 0.091832
Epoch [30/300], Loss: 0.082784
Epoch [40/300], Loss: 0.072293
Epoch [50/300], Loss: 0.062110
Epoch [60/300], Loss: 0.065546
Epoch [70/300], Loss: 0.058577
Epoch [80/300], Loss: 0.064985
Epoch [90/300], Loss: 0.057849
Epoch [100/300], Loss: 0.055324
Epoch [110/300], Loss: 0.058637
Epoch [120/300], Loss: 0.057290
Epoch [130/300], Loss: 0.056134
Epoch [140/300], Loss: 0.056673
Epoch [150/300], Loss: 0.057418
Epoch [160/300], Loss: 0.059509
Epoch [170/300], Loss: 0.055156
Epoch [180/300], Loss: 0.055951
Epoch [190/300], Loss: 0.059452
Epoch [200/300], Loss: 0.056339
Epoch [210/300], Loss: 0.049715
Epoch [220/300], Loss: 0.055623
Epoch [230/300], Loss: 0.053190
Epoch [240/300], Loss: 0.053942
Epoch [250/300], Loss: 0.055675
Epoch [260/300], Loss: 0.053847
Epoch [270/300], Loss: 0.055721
Epoch [280/300], Loss: 0.054503
Epoch [290/300], Loss: 0.055825
Epoch [300/300], Loss: 0.058228
Final Mean Squared Error: 2971435008.0
Sample Pre

<ipython-input-15-0d47887a9bc0>:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mobile_price_prediction_optimized.pth"))


Epoch [10/100], Loss: 3244520704.0000
Epoch [20/100], Loss: 3244517120.0000
Epoch [30/100], Loss: 3244513024.0000
Epoch [40/100], Loss: 3244507136.0000
Epoch [50/100], Loss: 3244498944.0000
Epoch [60/100], Loss: 3244486400.0000
Epoch [70/100], Loss: 3244469248.0000
Epoch [80/100], Loss: 3244444928.0000
Epoch [90/100], Loss: 3244412160.0000
Epoch [100/100], Loss: 3244369152.0000


Mean Squared Error on Test Set: 2610195200.0


Sample Predictions: [5.839687824249268, 1.7985594272613525, 3.744877815246582, 1.843387484550476, 2.2065060138702393]


<ipython-input-10-a114c1db9ac2>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mobile_price_prediction.pth"))


Epoch [10/300], Loss: 0.062176
Epoch [20/300], Loss: 0.047755
Epoch [30/300], Loss: 0.040718
Epoch [40/300], Loss: 0.040525
Epoch [50/300], Loss: 0.035260
Epoch [60/300], Loss: 0.031962
Epoch [70/300], Loss: 0.033902
Epoch [80/300], Loss: 0.032438
Epoch [90/300], Loss: 0.033478
Epoch [100/300], Loss: 0.032784
Epoch [110/300], Loss: 0.031391
Epoch [120/300], Loss: 0.032478
Epoch [130/300], Loss: 0.035414
Epoch [140/300], Loss: 0.034148
Epoch [150/300], Loss: 0.032543
Epoch [160/300], Loss: 0.031297
Epoch [170/300], Loss: 0.033277
Epoch [180/300], Loss: 0.031446
Epoch [190/300], Loss: 0.033010
Epoch [200/300], Loss: 0.031733
Epoch [210/300], Loss: 0.032428
Epoch [220/300], Loss: 0.032918
Epoch [230/300], Loss: 0.032033
Epoch [240/300], Loss: 0.031593
Epoch [250/300], Loss: 0.034145
Epoch [260/300], Loss: 0.033634
Epoch [270/300], Loss: 0.032550
Epoch [280/300], Loss: 0.032589
Epoch [290/300], Loss: 0.034554
Epoch [300/300], Loss: 0.032950
Final Mean Squared Error: 2835847424.0
Sample Pre

<ipython-input-14-0d47887a9bc0>:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("mobile_price_prediction_optimized.pth"))
